![logo](./img/TheBridge_RL.png)

# Taxi Autónomo (Smartcab), sin RL

## Contenidos

* [Definicion del Problema](#Definicion-del-Problema)  
* [Creando el entorno](#Creando-el-entorno)  
* [Sin aprendizaje por refuerzo](#Sin-aprendizaje-por-refuerzo)  
* [Visualizacion](#Visualizacion)  


### Definicion del Problema  
[al indice](#Contenidos)  


__Hay 4 ubicaciones (etiquetadas con diferentes letras) y nuestro trabajo es recoger al pasajero en una ubicación y dejarlo en otra.__

<img src="./img/Reinforcement_Learning_Taxi_Env.png" alt="drawing" width="600"/>


Recuerda, que las recompensas y castigos estan fijadas en este entorno de la siguiente manera:__   
__* Recibimos +20 puntos por un traslado exitoso.__  
__* Perdemos 1 punto por cada intervalo de tiempo que tarda.__  
__* También hay una penalización de 10 puntos por acciones de recogida y dejada ilegales.__  
__* Un movimiento que de lugar a un "choque" tiene la penalización de perder tiempo (el taxi no se desplaza pero se recibe la penalización de duración)__

### Creando el entorno  
[al indice](#Contenidos)  


In [ ]:
import gym

env = gym.make("Taxi-v3",render_mode="ansi").env
'''En esta línea, se crea un entorno de Gym llamado "Taxi-v3" y se almacena en la variable env. 
El argumento render_mode="ansi" se utiliza para configurar el modo de representación en el que se mostrará el entorno. 
El ".env" al final se utiliza para obtener el objeto del entorno real.'''

state,info = env.reset(seed=19)
'''Aquí, se reinicia el entorno utilizando el método reset() y se proporciona una semilla (seed) igual a 19 como argumento.'''

print(env.render())
'''Esta línea imprime el estado inicial del entorno utilizando el método render() del entorno Gym. 
El método render() muestra la representación visual del entorno en el modo especificado (en este caso, "ansi").'''

In [ ]:
movements = [2,0]

for action in movements:
    env.step(action)
    print(env.render())

Tenemos 6 posibles acciones (sur, norte, este, oeste, recoger, dejar) y 500 posibles estados (que combinan la posición del taxi, el punto de recogida, el de entrega, la situación del pasajero) codificados del 0 al 499

### Sin aprendizaje por refuerzo  
[al indice](#Contenidos)  


In [ ]:
timesteps = 0 
penalties, rewards = 0,0
simulated_time = 0
frames = []

done = False #se utilizará para controlar cuando se detiene el bucle
while not done:
    action= env.action_space.sample()
    '''En cada paso del bucle, se elige una acción aleatoria utilizando el método sample() del espacio de acciones 
    del entorno env. Esto simula la elección de una acción por parte de un agente de aprendizaje por refuerzo.'''
    if action < 4:
        simulated_time += 45
    else:
        simulated_time += 75
    '''Dependiendo de la acción elegida (menor que 4 o no), se incrementa el tiempo simulado simulated_time. 
    Esto representa que algunas acciones consumen más tiempo que otras en la simulación.'''
    state,reward,done,truncated,info = env.step(action) #Se ejecuta la acción elegida en el entorno env utilizando el método step(action)
    if reward == -10:
        penalties += 1
    rewards += reward
    '''Se actualizan las estadísticas. Si la recompensa es igual a -10, se incrementa el contador de penalizaciones 
    penalties, y se acumula la recompensa total rewards.'''
    frames.append({
        "frame":env.render(),
        "state": state,
        "action":action,
        "reward":reward,
        "elapsed":simulated_time  
    })
    '''Se registra información relevante en el paso actual en la lista frames. Esto incluye el fotograma (representación visual)
    del entorno, el estado, la acción, la recompensa y el tiempo simulado en ese momento.'''
    timesteps += 1
    '''Se incrementa el contador de timesteps para llevar un registro del número de pasos en la simulación.'''

#UNA VEZ FINALIZA EL BUCLE, SE IMPRIMEN LAS ESTADÍSTICAS FINALES
print("Timesteps:",timesteps)
print("Penalties:",penalties)
print("Tiempo real simulado:",simulated_time)

En tiempo de ejecución de código no ha sido nada... Pero imagina por un momento que hubiera sido una prueba en un entorno real... Recuerda que esta es una de las razones por las que se usan entornos simulados para aprender. Aunque no siempre es posible o recomendable (los coches autónomos de verdad aprenden sobre el terreno)

### Visualizacion  
[al indice](#Contenidos)  


Para poder ver qué es lo que realmente está haciendo el "smart"cab, vamos a crearnos una función que pinte cada frame, dejando un intervalo de tiempo entre frame y frame para crear el efecto de animación.

In [ ]:
# Importamos dos comandos necesarios para limpiar la pantalla y para generar el intervalo entre frames
from IPython.display import clear_output 
from time import sleep

def episode_animation(frames):
    for i, frame in enumerate(frames): # Recorremos todo el conjunto de frames
        clear_output(wait=True) # Limpiamos la "pantalla"
        print(frame['frame']) # Visualizamos el "pantallazo" resultado de cada acción
        print(f"Timestep: {i + 1}") # Aumentamos el contador de pasos/steps
        # Imprimimos el resto de valores correspondientes a cada frame y que hemos guardado al realizar el "aprendizaje"
        print(f"State: {frame['state']}") 
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        print(f"Elapsed time (sec.): {frame['elapsed']}")
        sleep(.1) # "Dormimos" el programa un tiempo para que nuestro ojo pueda ver la imagen antes de borrarla y mostrar la siguiente
        


In [ ]:
episode_animation(frames[max(0,len(frames)-500):]) # Lanzamos la animacion, pero asegurándonos de que sólo mostramos como mucho 500 frames. Si quieres verla entera (para gustos los colores...), sólo tienes que quitar indexado y volver a ejecutar.

Cómo puedes comprobar este mecanimo, aunque cumple con el cometido, tiene severas limitaciones:
- Los tiempos son inaceptables
- No se guarda el "aprendizaje", pero aunque lo hiciéramos, sólo valdría para la situación de partida (el taxi en la posición 3,1. Recoger en Y entregar en R)

Necesitamos un mecanismo de mejora de la estrategia que no sólo reduzca los tiempos y los optimice al máximo, sino que además sea flexible como para poder aplicarse en cualquier situación. Ese es el objetivo de la siguiente sesión: Q-Learning.

## Ahora tú

Aunque a continuación tienes un ejercicio específico, ¿por qué no inventas una política más acertada para nuestro SmartCab? Por ejemplo puedes jugar con la variable info, ya que eso no es hacer trampa. Nuestro SmartCab tendría su LIDAR y otros sensores para detectar paredes y cuando se acaba el terreno por ejemplo. Es decir viendo la action_mask podría evitarse acciones "inútiles"